In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency, norm
from data_viz import *
from hypothesis_functions import *

In [2]:
data = pd.read_csv("data/animal_center.csv")
data

,animal_id,name,animal_type,sex_intake,sex_outcome,breed,color,date_of_birth,found_location,intake_datetime,outcome_datetime,intake_type,intake_condition,outcome_type,outcome_subtype,age_upon_intake(years),age_upon_outcome(years),duration(days)
0,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research in Austin (TX),2014-03-07 14:26:00,2014-03-08 17:10:00,Public Assist,Normal,Return to Owner,NaN,6.7,6.7,1.0
1,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research Blvd in Austin (TX),2014-12-19 10:21:00,2014-12-20 16:35:00,Public Assist,Normal,Return to Owner,NaN,7.4,7.4,1.0
2,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,Colony Creek And Hunters Trace in Austin (TX),2017-12-07 14:07:00,2017-12-07 00:00:00,Stray,Normal,Return to Owner,NaN,10.4,10.4,-1.0
3,A047759,Oreo,Dog,Neutered Male,Neutered Male,Dachshund,Tricolor,2004-04-02,Austin (TX),2014-04-02 15:55:00,2014-04-07 15:12:00,Owner Surrender,Normal,Transfer,Partner,10.0,10.0,4.0
4,A134067,Bandit,Dog,Neutered Male,Neutered Male,Shetland Sheepdog,Brown/White,1997-10-16,12034 Research Blvd in Austin (TX),2013-11-16 09:02:00,2013-11-16 11:54:00,Public Assist,Injured,Return to Owner,NaN,16.1,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157621,A894189,Kittler,Cat,Intact Female,NaN,Domestic Shorthair Mix,Black/White,NaN,Austin (TX),2023-12-04 17:16:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157622,A894190,Zeeb,Cat,Intact Male,NaN,Domestic Shorthair Mix,Brown Tabby,NaN,Austin (TX),2023-12-04 17:16:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157623,A894191,Lucky,Cat,Intact Male,NaN,Domestic Shorthair Mix,Gray/White,NaN,Austin (TX),2023-12-04 17:16:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN
157624,A894192,Lil Bit,Cat,Intact Female,NaN,Domestic Shorthair Mix,Black/White,NaN,Austin (TX),2023-12-04 17:16:00,NaN,Owner Surrender,Normal,NaN,NaN,NaN,NaN,NaN


In [3]:
data.columns

Index(['animal_id', 'name', 'animal_type', 'sex_intake', 'sex_outcome',
       'breed', 'color', 'date_of_birth', 'found_location', 'intake_datetime',
       'outcome_datetime', 'intake_type', 'intake_condition', 'outcome_type',
       'outcome_subtype', 'age_upon_intake(years)', 'age_upon_outcome(years)',
       'duration(days)'],
      dtype='object')

In [4]:
data.outcome_type.value_counts()

Adoption           73875
Transfer           44903
Return to Owner    24429
Euthanasia          9965
Died                1470
Rto-Adopt           1079
Disposal             767
Missing               84
Relocate              26
Stolen                 5
Name: outcome_type, dtype: int64

In [5]:
no_kill_research = data[["animal_id", "intake_datetime", "outcome_datetime", "outcome_type", "duration(days)"]].copy()
no_kill_research

,animal_id,intake_datetime,outcome_datetime,outcome_type,duration(days)
0,A006100,2014-03-07 14:26:00,2014-03-08 17:10:00,Return to Owner,1.0
1,A006100,2014-12-19 10:21:00,2014-12-20 16:35:00,Return to Owner,1.0
2,A006100,2017-12-07 14:07:00,2017-12-07 00:00:00,Return to Owner,-1.0
3,A047759,2014-04-02 15:55:00,2014-04-07 15:12:00,Transfer,4.0
4,A134067,2013-11-16 09:02:00,2013-11-16 11:54:00,Return to Owner,0.0
...,...,...,...,...,...
157621,A894189,2023-12-04 17:16:00,NaN,NaN,NaN
157622,A894190,2023-12-04 17:16:00,NaN,NaN,NaN
157623,A894191,2023-12-04 17:16:00,NaN,NaN,NaN
157624,A894192,2023-12-04 17:16:00,NaN,NaN,NaN


In [6]:
no_kill_research["intake_datetime"] = pd.to_datetime(no_kill_research["intake_datetime"])
no_kill_research["outcome_datetime"] = pd.to_datetime(no_kill_research["outcome_datetime"])

no_kill_research["intake_y_m"] = no_kill_research["intake_datetime"].dt.to_period("M").astype(str)
no_kill_research["outcome_y_m"] = no_kill_research["outcome_datetime"].dt.to_period("M").astype(str)

no_kill_research["intake_yr"] = no_kill_research["intake_datetime"].dt.to_period("Y").astype(str)
no_kill_research["outcome_yr"] = no_kill_research["outcome_datetime"].dt.to_period("Y").astype(str)

no_kill_research["intake_m"] = pd.to_datetime(no_kill_research["intake_datetime"], format='%m').dt.month_name()
no_kill_research["outcome_m"] = pd.to_datetime(no_kill_research["outcome_datetime"], format='%m').dt.month_name()

no_kill_research

,animal_id,intake_datetime,outcome_datetime,outcome_type,duration(days),intake_y_m,outcome_y_m,intake_yr,outcome_yr,intake_m,outcome_m
0,A006100,2014-03-07 14:26:00,2014-03-08 17:10:00,Return to Owner,1.0,2014-03,2014-03,2014,2014,March,March
1,A006100,2014-12-19 10:21:00,2014-12-20 16:35:00,Return to Owner,1.0,2014-12,2014-12,2014,2014,December,December
2,A006100,2017-12-07 14:07:00,2017-12-07 00:00:00,Return to Owner,-1.0,2017-12,2017-12,2017,2017,December,December
3,A047759,2014-04-02 15:55:00,2014-04-07 15:12:00,Transfer,4.0,2014-04,2014-04,2014,2014,April,April
4,A134067,2013-11-16 09:02:00,2013-11-16 11:54:00,Return to Owner,0.0,2013-11,2013-11,2013,2013,November,November
...,...,...,...,...,...,...,...,...,...,...,...
157621,A894189,2023-12-04 17:16:00,NaT,NaN,NaN,2023-12,NaT,2023,NaT,December,NaN
157622,A894190,2023-12-04 17:16:00,NaT,NaN,NaN,2023-12,NaT,2023,NaT,December,NaN
157623,A894191,2023-12-04 17:16:00,NaT,NaN,NaN,2023-12,NaT,2023,NaT,December,NaN
157624,A894192,2023-12-04 17:16:00,NaT,NaN,NaN,2023-12,NaT,2023,NaT,December,NaN


## **Group by year-month**

In [7]:
no_kill_by_ym = no_kill_rate(no_kill_research, "intake_y_m", "outcome_y_m")
no_kill_by_ym.head(50)

,time,total_intake,total_outcome,euthanasia_cases,total_died_cases,save_rate(total died),save_rate(euthanasia)
0,2013-10,1589.0,1126,147,162,89.804909,90.748899
1,2013-11,1323.0,1213,120,129,90.249433,90.929705
2,2013-12,1270.0,1431,163,171,86.535433,87.165354
3,2014-01,1271.0,1224,114,124,90.243902,91.030685
4,2014-02,1192.0,1115,144,155,86.996644,87.919463
5,2014-03,1484.0,1479,267,298,79.919137,82.008086
6,2014-04,1535.0,1399,147,163,89.381107,90.423453
7,2014-05,1957.0,1625,156,178,90.904446,92.028615
8,2014-06,1793.0,1751,167,184,89.737869,90.686001
9,2014-07,1888.0,2070,165,194,89.724576,91.260593


In [8]:
fig = px.line(no_kill_by_ym,
              x="time",
              y=["save_rate(total died)", "save_rate(euthanasia)"])
fig.update_traces()
fig.show()

## **Group by year**

In [9]:
no_kill_by_yr = no_kill_rate(no_kill_research, "intake_yr", "outcome_yr")
no_kill_by_yr

,time,total_intake,total_outcome,euthanasia_cases,total_died_cases,save_rate(total died),save_rate(euthanasia)
0,2013,4182.0,3770,430,462,88.952654,89.717838
1,2014,18654.0,18559,1844,2080,88.849576,90.114721
2,2015,18709.0,18485,1461,1684,90.998984,92.190924
3,2016,17673.0,17662,1161,1434,91.885928,93.430657
4,2017,17559.0,17643,1083,1327,92.442622,93.832223
5,2018,16975.0,16734,808,1036,93.896907,95.240059
6,2019,19726.0,19769,788,1127,94.286728,96.005272
7,2020,9585.0,9772,698,892,90.693792,92.717788
8,2021,12040.0,11962,597,789,93.446844,95.041528
9,2022,11891.0,11882,607,770,93.524514,94.895299


In [10]:
fig = px.line(no_kill_by_yr,
              x="time",
              y=["save_rate(total died)", "save_rate(euthanasia)"])
fig.show()

## **Group by month**

In [13]:
no_kill_by_m = no_kill_rate(no_kill_research, "intake_m", "outcome_m")
month_order = ["January", "February", "March", "April",
               "May", "June", "July", "August",
               "September", "October", "November", "December"]
no_kill_by_m["time"] = pd.Categorical(no_kill_by_m["time"],
                                      categories=month_order,
                                      ordered=True)
no_kill_by_m = no_kill_by_m.sort_values("time")
no_kill_by_m

,time,total_intake,total_outcome,euthanasia_cases,total_died_cases,save_rate(total died),save_rate(euthanasia)
4,January,10831,11418,555,660,93.906380,94.875819
3,February,9835,9974,679,776,92.109812,93.096085
7,March,11944,11685,1427,1692,85.833891,88.052579
0,April,12479,11213,929,1080,91.345460,92.555493
8,May,16165,13519,913,1140,92.947727,94.351995
6,June,15733,15093,1042,1332,91.533719,93.376978
5,July,14429,15329,820,1096,92.404186,94.317000
1,August,13990,14991,726,982,92.980701,94.810579
11,September,13727,13670,686,881,93.581992,95.002550
10,October,14931,14179,907,1110,92.565803,93.925390


In [14]:
fig = px.line(no_kill_by_m,
              x="time",
              y=["save_rate(total died)", "save_rate(euthanasia)"])
fig.show()

In [12]:
fig = px.line(no_kill_by_ym,
              x="time",
              y=["total_intake", "total_outcome"])
fig.show()